In [2]:
!pip install pymysql



  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 44.8/44.8 kB 1.1 MB/s eta 0:00:00


In [20]:
import pymysql
import csv


csv_files = {
    'members':'member.csv',
    'staff': 'staff.csv',
    'books': 'book.csv',
    'catalog': 'catalog - catalog.csv.csv',
    'borrows': 'borrows.csv',
    'membership': 'membership.csv'
}

# SQL insert statements for each table
insert_statements = {
    'members': "INSERT INTO members (ssn,name,home_address,campus_address,phone_no) VALUES (%s, %s, %s, %s, %s)",
    'staff': "INSERT INTO staff (ssn, name, position) VALUES (%s, %s, %s)",
    'books': "INSERT INTO books (isbn, title,language,copy_no,edition) VALUES (%s, %s, %s, %s, %s)",
    'catalog': "INSERT INTO catalog (isbn, title,author,description) VALUES (%s, %s, %s,%s)",
    'borrows': "INSERT INTO borrows (ssn, isbn, name, borrow_date,due_date, return_date) VALUES (%s, %s, %s, %s, %s,%s)",
    'membership': "INSERT INTO membership (ssn, name, card_no, expiry_date, type) VALUES (%s, %s, %s, %s, %s)"
}

# Function to load data from a CSV file and insert into the database
def load_data(table_name, csv_path, insert_sql):
    connection = pymysql.connect(host='localhost', user='root', password='Cherukuri01@', database='library')
    try:
        with connection.cursor() as cursor:
            with open(csv_path, newline='', encoding='utf-8') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    cursor.execute(insert_sql, row)
        connection.commit()
    finally:
        connection.close()

# Execute the data loading for each table
for table, path in csv_files.items():
    load_data(table, path, insert_statements[table])
    print(f"Data loaded into {table} from {path}")

Data loaded into membership from membership.csv


In [3]:
import pymysql
from datetime import datetime

def connect_to_db():
    return pymysql.connect(host='localhost', user='root', password='Cherukuri01@', db='library')

def add_new_member():
    ssn = input("Enter member ssn: ")
    name = input("Enter member name: ")
    home_address= input("Enter member home_address: ")
    campus_address= input("Enter member campus_address: ")
    phone_no= input("Enter member phone_no: ")
    with connect_to_db() as connection:
        with connection.cursor() as cursor:
            sql = "INSERT INTO members (ssn,name,home_address,campus_address,phone_no) VALUES (%s, %s, %s, %s, %s)"
            cursor.execute(sql, (ssn,name,home_address,campus_address,phone_no))
        connection.commit()
    print("New member added.")

def add_new_book():
    isbn = input("Enter ISBN: ")
    title = input("Enter book title: ")
    language = input("Enter language: ")
    copy_no= input("Enter copy_no")
    edition=input("Enter edition")
    
    with connect_to_db() as connection:
        with connection.cursor() as cursor:
            sql = "INSERT INTO books (isbn, title,language,copy_no,edition) VALUES (%s, %s, %s, %s, %s)"
            cursor.execute(sql, (isbn, title,language,copy_no,edition))
        connection.commit()
    print("New book added.")

def add_new_borrow():
    ssn = input("Enter member ssn: ")
    isbn = input("Enter ISBN of the book to borrow: ")
    name = input("Enter name: ")
    #borrow_date = datetime.now().strftime('%Y-%m-%d')
    borrow_date = input("Enter borrow date (YYYY-MM-DD): ")
    due_date = input("Enter due date (YYYY-MM-DD): ")
    return_date= None
    with connect_to_db() as connection:
        with connection.cursor() as cursor:
            sql = "INSERT INTO borrows (ssn, isbn, name, borrow_date,due_date, return_date) VALUES (%s, %s, %s, %s, %s,%s)"
            cursor.execute(sql, (ssn, isbn, name, borrow_date,due_date, return_date))
        connection.commit()
    print("Borrow transaction added.")

def handle_return():
    isbn = input("Enter ISBN of the book being returned: ")
    return_date = datetime.now().strftime('%Y-%m-%d')
    with connect_to_db() as connection:
        with connection.cursor() as cursor:
            sql = "UPDATE borrows SET return_date = %s WHERE isbn = %s AND return_date IS NULL"
            cursor.execute(sql, (return_date, isbn))
            sql_select = """
            SELECT b.isbn, b.name, b.ssn, b.borrow_date, b.return_date, DATEDIFF(b.return_date, b.borrow_date) AS days_borrowed
            FROM borrows b WHERE b.isbn = %s AND b.return_date = %s;
            """
            cursor.execute(sql_select,(isbn,return_date))
            book_info = cursor.fetchone()
            columns = [desc[0] for desc in cursor.description]
    
            for col, value in zip(columns, book_info):
                print(f"'{col}':'{value}'")
                    
            

def renew_membership():
    ssn= input("Enter ssn: ")
    new_expiry_date = input("Enter new expiry date (YYYY-MM-DD): ")
    with connect_to_db() as connection:
        with connection.cursor() as cursor:
            sql = "UPDATE membership SET expiry_date = %s WHERE ssn = %s"
            cursor.execute(sql, (new_expiry_date, ssn))
        connection.commit()
    print("Membership renewed.")

def main():
    options = {
        '1': add_new_member,
        '2': add_new_book,
        '3': add_new_borrow,
        '4': handle_return,
        '5': renew_membership
    }
    while True:
        print("\nChoose an option:")
        print("1. Add a new member")
        print("2. Add a new book")
        print("3. Add a new borrow (loan)")
        print("4. Handle the return of a book")
        print("5. Renew membership")
        print("0. Exit")
        choice = input("Enter your choice: ")
        if choice == '0':
            break
        if choice in options:
            options[choice]()
        else:
            print("Invalid choice, please try again.")

if __name__ == '__main__':
    main()


Choose an option:
1. Add a new member
2. Add a new book
3. Add a new borrow (loan)
4. Handle the return of a book
5. Renew membership
0. Exit
Enter your choice: 1
Enter member ssn: 987654320
Enter member name: sharath
Enter member home_address: 1000 dr
Enter member campus_address: 1000 dr
Enter member phone_no: 1234567877
New member added.

Choose an option:
1. Add a new member
2. Add a new book
3. Add a new borrow (loan)
4. Handle the return of a book
5. Renew membership
0. Exit
Enter your choice: 3
Enter member ssn: 987654320
Enter ISBN of the book to borrow: 9780061120084
Enter name: sharath
Enter borrow date (YYYY-MM-DD): 2024-03-25
Enter due date (YYYY-MM-DD): 2024-04-7
Borrow transaction added.

Choose an option:
1. Add a new member
2. Add a new book
3. Add a new borrow (loan)
4. Handle the return of a book
5. Renew membership
0. Exit
Enter your choice: 5
Enter ssn: 111222333
Enter new expiry date (YYYY-MM-DD): 2024-04-14
Membership renewed.

Choose an option:
1. Add a new membe

In [6]:
from datetime import datetime

def connect_to_db():
    return pymysql.connect(host='localhost', user='root', password='Cherukuri01@', db='library')

def notify_overdue_books():
    today = datetime.now().strftime('%Y-%m-%d')
    with connect_to_db() as connection:
        with connection.cursor() as cursor:
            query = """
            SELECT b.isbn, b.name FROM borrows b WHERE b.due_date < %s AND b.return_date IS NULL;
            """
            cursor.execute(query, (today,))
            results = cursor.fetchall()
            
            if results:
                print("Overdue Book Notifications:")
                for row in results:
                    print("overdue info--isbn,name:",row)
            else:
                print("No overdue books found.")

def notify_membership_renewal():
    today = datetime.now().strftime('%Y-%m-%d')
    with connect_to_db() as connection:
        with connection.cursor() as cursor:
            query = """
            SELECT name, card_no FROM membership WHERE expiry_date <= %s;
            """
            cursor.execute(query, (today,))
            results = cursor.fetchall()
            
            if results:
                print("Membership Renewal Notifications:")
                for row in results:
                    print("renewal info--name,card_no:",row)
            else:
                print("No membership renewals needed soon.")

def main():
    notify_overdue_books()
    notify_membership_renewal()

if __name__ == '__main__':
    main()

Overdue Book Notifications:
overdue info--isbn,name: (9780061120084, 'sharath')
Membership Renewal Notifications:
renewal info--name,card_no: ('Alice Johnson', 345678)
